In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU available")

In [ ]:
import cv2
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
!pip install /kaggle/input/mtcnn1/mtcnn-0.1.0-py3-none-any.whl

In [ ]:
import json

with open("/kaggle/input/deepfake-detection-challenge/train_sample_videos/metadata.json",'r') as f:
    file=json.load(f)

video_name=[]
label=[]
original=[]
for key in file.keys():
    video_name.append(key)
    label.append(file[key]["label"])
    

df=pd.DataFrame({"video_name":video_name,"label":label})

In [ ]:
df.head()

In [ ]:
df["label"].value_counts()

In [ ]:
os.mkdir(os.path.join("/kaggle/working/","video_images"))

real=df[df["label"]=="REAL"]

for ele in real["video_name"][:5]:
    print(ele)

len(real)

#print("aagfhgtpmv.mp4" in fake["video_name"].tolist())

def get_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frames.append(frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    cap.release()
    
    facial_key=[]
    for i in sorted([0,5,10,15,20,25,29,40,42,24,56,245,167,199,99,69,50,70,80,100,120,140,150,180,190,200,210,220,240,270,280,290,299,111,222,11]):      #  change the number for label real and fake
        if i<len(frames):
            frame=frames[i]
            frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
            facial_key.append(frame)
    return facial_key


!pip install /kaggle/input/facenet/facenet_pytorch-2.2.9-py3-none-any.whl

from facenet_pytorch import MTCNN



detector = MTCNN(margin=20, keep_all=True, post_process=False, device=device)
def detect_face(facial_key):
    images = []
    for key in facial_key:
        faces, prob = detector.detect(key)
        
        # Ensure faces and prob are not None
        if faces is not None and prob is not None:
            for face, confidence_score in zip(faces, prob):
                if face is not None and confidence_score >= 0.9:
                    x1, y1, x2, y2 = face.astype(int)  # Convert to integer for bounding box coordinates
                    # Adjust bounding box with margin
                    x1 = max(0, x1)
                    y1 = max(0, y1)
                    x2 = min(key.shape[1], x2)
                    y2 = min(key.shape[0], y2)
                    images.append(key[y1:y2, x1:x2])
    return images

def get_images(train_dir,final_dir,fake_df):
    for video in os.listdir(train_dir):
        if os.path.exists(os.path.join(final_dir,video)):
            continue
        if video not in fake_df:
            continue
        os.mkdir(os.path.join(final_dir,video))
        
        facial_key=get_frames(os.path.join(train_dir,video))
        
        faces=detect_face(facial_key)
        
        for i in range(len(faces)):
            path=os.path.join(final_dir,video,str(i)+".png")
            face=cv2.cvtColor(faces[i],cv2.COLOR_BGR2RGB)
            cv2.imwrite(path, face, [cv2.IMWRITE_PNG_COMPRESSION, 0])

In [ ]:
get_images("/kaggle/input/deepfake-detection-challenge/train_sample_videos","/kaggle/working/video_images",real["video_name"].tolist())

In [ ]:
print(len(next(os.walk('/kaggle/working/video_images'))[1]))

In [ ]:
import os
os.chdir(r'/kaggle/working')

!tar -czf real.tar.gz video_images

from IPython.display import FileLink

FileLink(r'real.tar.gz')

In [ ]:
video_path="/kaggle/input/deepfake-detection-challenge/train_sample_videos/aagfhgtpmv.mp4"
cap = cv2.VideoCapture(video_path)
#cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
#fps = cap.get(cv2.CAP_PROP_FPS)
#frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret==True:
        frames.append(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()

len(frames)

facial_key=[]
for i in [0,50,100,150,200,250]:
    frame=frames[i]
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    facial_key.append(frame)

In [ ]:
imgplot = plt.imshow(facial_key[0])
plt.show()

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
locations = face_cascade.detectMultiScale(cv2.cvtColor(facial_key[0],cv2.COLOR_RGB2BGR))

In [ ]:
for face_location in locations:

    # Print the location of each face in this image
    x, y, w, h = face_location
    print("A face is located at pixel location X: {}, Y: {}, Width: {}, Height: {}".format(x, y, w, h))

    # You can access the actual face itself like this:
    face_image = facial_key[0][y:y+h, x:x+w]
    
    print(face_image.shape)
    fig, ax = plt.subplots(1,1, figsize=(5, 5))
    plt.grid(False)
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)
    ax.imshow(face_image)

In [ ]:
import mtcnn
print(mtcnn.__version__)

from mtcnn.mtcnn import MTCNN

detector = MTCNN()

faces = detector.detect_faces(facial_key[3])
faces

image=[]
print(len(faces))
for face in faces:
    x,y,w,h = face['box']

    x_top=int(x-w/2)
    y_top=int(y-h/2)
    #rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
    image.append(facial_key[3][y:y+h,x:x+w])
    
     
    # Add the patch to the Axes
    #ax.add_patch(rect)

plt.imshow(image[1])


plt.imshow(image[1])

In [ ]:
def isotropic_resize(image,size,resample=cv2.INTER_AREA):
    
    h,w=image.shape[:2]
    if max(w,h)==size:
        return image
    if w>h:
        h=h*size//w
        w=size
    else:
        w=w*size//h
        h=size
    resized=cv2.resize(image,(w,h),interpolation=resample)
    return resized

def make_square(image):
    h,w=image.shape[:2]
    size=max(h,w)
    t=0
    b=size-h
    l=0
    r=size-w
    
    return cv2.copyMakeBorder(image,t,b,l,r,cv2.BORDER_CONSTANT,value=0)

In [ ]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:

mean=[0.485,0.456,0.405]
std=[0.229,0.224,0.225]
size=224
transform=transforms.Compose([
    transforms.Lambda(lambda img:cv2.cvtColor(img,cv2.COLOR_BGR2RGB)),
    transforms.Lambda(lambda img:isotropic_resize(img,size)),
    transforms.Lambda(lambda img:make_square(img)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean,std=std)
])

In [ ]:
class_0_dir=os.path.join("/kaggle/input/deepfakeimages/Images","real_images")
class_1_dir=os.path.join("/kaggle/input/deepfakeimages/Images","Fake_images")    
fake_images=[]
for subdir in os.listdir(class_1_dir):
    subdir_path=os.path.join(class_1_dir,subdir)
    fake_images.extend([os.path.join(subdir_path,img) for img in os.listdir(subdir_path)])
real_images=[]
for subdir in os.listdir(class_0_dir):
    subdir_path=os.path.join(class_0_dir,subdir)
    real_images.extend([os.path.join(subdir_path,img) for img in os.listdir(subdir_path)])
images=[(img,0) for img in real_images]+[(img,1) for img in fake_images]

In [ ]:
len(images)

In [ ]:
class ImageDataset(Dataset):
    
    def __init__(self,images,transform=None):
        
        self.images=images
        self.transform=transform
    
    def __len__(self):
        return len(self.images)
    
    
    def __getitem__(self,idx):
        
        image_path,label=self.images[idx]
        img=cv2.imread(image_path)
        if self.transform:
            img=self.transform(img)
        
        return cv2.cvtColor(img,cv2.COLOR_BGR2RGB),label

In [ ]:
import random

random.seed(42)

random.shuffle(images)

train_size=int(0.8*len(images))

train_images=images[:train_size]

val_images=images[train_size:]

In [ ]:
train_dataset=ImageDataset(train_images)
val_datasset=ImageDataset(val_images,transform)

In [ ]:
# Get the image tensor from the dataset
img_tensor = train_dataset[0][0]

# Convert the image tensor to a NumPy array
#img_np = img_tensor.numpy()

# Transpose the array to match the shape expected by matplotlib (C, H, W)
#img_np = np.transpose(img_np, (1, 2, 0))

# Display the image
plt.imshow(img_tensor)
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
images[0]

In [ ]:
plt.figure(figsize=(20, 10))
for i in range(50):
    plt.subplot(5, 10, i+1)
    img = plt.imread(images[i][0])
    plt.imshow(img)
    plt.axis('off')

plt.show()